In [ ]:
MODEL_PATH = "clinical_trial_model.keras"
MODEL_WEIGHTS_PATH = "clinical_trial_model.weights.h5"

# Data

In [ ]:
import numpy as np
from random import randint
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler

In [ ]:
train_samples = []  # age
train_labels = []  # 0: no side effects, 1: side effects

for i in range(50):
    # The ~5% of younger individuals who did experience side effects
    random_younger = randint(13, 64)
    train_samples.append(random_younger)
    train_labels.append(1)

    # The ~5% of older individuals who did not experience side effects
    random_older = randint(65, 100)
    train_samples.append(random_older)
    train_labels.append(0)

for i in range(1000):
    # The ~95% of younger individuals who did not experience side effects
    random_younger = randint(13, 64)
    train_samples.append(random_younger)
    train_labels.append(0)

    # The ~95% of older individuals who did experience side effects
    random_older = randint(65, 100)
    train_samples.append(random_older)
    train_labels.append(1)

In [ ]:
train_samples = np.array(train_samples)
train_labels = np.array(train_labels)
train_samples, train_labels = shuffle(train_samples, train_labels)

scaler = MinMaxScaler(feature_range=(0, 1))  # Scale the data to be between 0 and 1
scaled_train_samples = scaler.fit_transform(train_samples.reshape(-1, 1))

In [ ]:
test_samples = []  # age
test_labels = []  # 0: no side effects, 1: side effects

for i in range(10):
    random_younger = randint(13, 64)
    test_samples.append(random_younger)
    test_labels.append(1)

    random_older = randint(65, 100)
    test_samples.append(random_older)
    test_labels.append(0)

for i in range(200):
    random_younger = randint(13, 64)
    test_samples.append(random_younger)
    test_labels.append(0)

    random_older = randint(65, 100)
    test_samples.append(random_older)
    test_labels.append(1)

In [ ]:
test_samples = np.array(test_samples)
test_labels = np.array(test_labels)
test_samples, test_labels = shuffle(test_samples, test_labels)

scaled_test_samples = scaler.fit_transform(test_samples.reshape(-1, 1))

# Simple tf.keras Sequential Model

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential  # type: ignore
from tensorflow.keras.layers import Dense, Activation  # type: ignore
from tensorflow.keras.optimizers import Adam  # type: ignore
from tensorflow.keras.metrics import categorical_crossentropy  # type: ignore

In [ ]:
model = Sequential(
    [
        Dense(units=16, input_shape=(1,), activation="relu"),
        Dense(units=32, activation="relu"),
        Dense(units=2, activation="softmax"),
    ]
)

In [ ]:
model.fit(
    scaled_train_samples,
    train_labels,
    validation_split=0.1,
    batch_size=10,
    epochs=30,
    shuffle=True,
    verbose=2,
)

In [ ]:
predictions = model.predict(scaled_test_samples, batch_size=10, verbose=0)

In [ ]:
rounded_predictions = np.argmax(predictions, axis=-1)
rounded_predictions

## Confusion Matrix

In [ ]:
%matplotlib inline
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

In [ ]:
cm = confusion_matrix(y_true=test_labels, y_pred=rounded_predictions)

In [ ]:
def plot_confusion_matrix(
    cm, classes, normalize=False, title="Confusion Matrix", cmap=plt.cm.Blues
):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation="nearest", cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print("Confusion matrix, without normalization")

    print(cm)

    thresh = cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(
            j,
            i,
            cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black",
        )

    plt.tight_layout()
    plt.ylabel("True label")
    plt.xlabel("Predicted label")

In [ ]:
cm_plot_labels = ["no_side_effects", "had_side_effects"]
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title="Confusion Matrix")

## Save
### 1. model.save()

In [ ]:
import os.path

if not os.path.isfile(MODEL_PATH):
    model.save(MODEL_PATH)

In [ ]:
from tensorflow.keras.models import load_model  # type: ignore

new_model = load_model(MODEL_PATH)
new_model.summary()
new_model.get_weights()
# new_model.optimizer # The tutorial output mentioned the Adam optimizer, but here the output is keras.src.optimizers.rmsprp.RMSprop object

### 2. model.to_json() / model.to_yaml()

In [ ]:
json_string = model.to_json()
# yaml_string = model.to_yaml()
json_string

In [ ]:
from tensorflow.keras.models import model_from_json  # type: ignore

# from tensorflow.keras.models import model_from_yaml  # type: ignore

new_architecture = model_from_json(json_string)
# new_architecture = model_from_yaml(yaml_string)
new_architecture.summary()

### 3. model.save_weights()

In [ ]:
if not os.path.isfile(MODEL_WEIGHTS_PATH):
    model.save_weights(MODEL_WEIGHTS_PATH)

In [ ]:
model2 = Sequential(
    [
        Dense(16, input_shape=(1,), activation="relu"),
        Dense(32, activation="relu"),
        Dense(2, activation="softmax"),
    ]
)

model2.load_weights(MODEL_WEIGHTS_PATH)
model2.get_weights()